# Exercise - Neural Network

The data set for this exercise is from the banking industry. It contains data about the home loans of 2,500 bank clients. Each row represents a single loan. The columns include the characteristics of the client who used a loan. This is a binary classification task: predict whether a loan will be bad or not (1=Yes, 0=No). This is an important task for banks to prevent bad loans from being issued.

## Description of Variables

The description of variables are provided in "Loan - Data Dictionary.docx"

## Goal

Use the **loan.csv** data set and build a model to predict **BAD**. 



# Read and Prepare the Data

In [1]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [2]:
#We will predict the "price" value in the data set:

loan = pd.read_csv("loan.csv")
loan.head()

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,0,25900,61064.0,94714.0,DebtCon,Office,2.0,0.0,0.0,98.809375,0.0,23.0,34.565944
1,0,26100,113266.0,182082.0,DebtCon,Sales,18.0,0.0,0.0,304.852469,1.0,31.0,33.193949
2,1,50000,220528.0,300900.0,HomeImp,Self,5.0,0.0,0.0,0.000000,0.0,2.0,NaN
3,1,22400,51470.0,68139.0,DebtCon,Mgr,9.0,0.0,0.0,31.168696,2.0,8.0,37.952180
4,0,20900,62615.0,87904.0,DebtCon,Office,5.0,NaN,NaN,177.864849,NaN,15.0,36.831076


# Split data (train/test)

In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(loan, test_size=0.3)

# Data Prep

Perform your data prep here. You can use pipelines like we do in the tutorials. Otherwise, feel free to use your own data prep steps. Eventually, you should do the following at a minimum:<br>
- Separate inputs from target<br>
- Impute/remove missing values<br>
- Standardize the continuous variables<br>
- One-hot encode categorical variables<br>

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

## Separate the target variable 

In [5]:
train_target = train['BAD']
test_target = test['BAD']

train_inputs = train.drop(['BAD'], axis=1)
test_inputs = test.drop(['BAD'], axis=1)

## Feature Engineering: Derive a new column

Examples:
- Ratio of delinquent to total number of credit lines
- Ratio of loan to value of current property
- Convert yr_renovated to a binary variable (i.e., renovated or not)
- (etc.)

In [6]:
def new_col(df):
    
    #Create a copy so that we don't overwrite the existing dataframe
    df1 = df.copy()

    # Use the formula, though fill in 0s when the value is 0/0 (because 0/0 generates "nan" values)
    df1['ratio_D_C'] = (df1['DELINQ']/df1['CLNO']).fillna(0)

    # Replace the infinity values with 1 (because a value divided by 0 generates infinity)
    df1['ratio_D_C'].replace(np.inf, 1, inplace=True)

    return df1[['ratio_D_C']]
    # You can use this to check whether the calculation is made correctly:
    #return df1
    

In [8]:
#Let's test the new function:

# Send the train set to the function we created
new_col(train)


,ratio_D_C
1552,0.000000
2290,0.000000
1398,0.000000
1775,0.035714
2299,0.000000
...,...
1638,0.125000
1095,0.181818
1130,0.000000
1294,0.000000


##  Identify the numeric, binary, and categorical columns

In [9]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [10]:
numeric_columns

['LOAN',
 'MORTDUE',
 'VALUE',
 'YOJ',
 'DEROG',
 'DELINQ',
 'CLAGE',
 'NINQ',
 'CLNO',
 'DEBTINC']

In [11]:
categorical_columns

['REASON', 'JOB']

In [22]:
# define your feature engineered column(s) here
feat_eng_columns = ['DELINQ','CLNO']

# Pipeline

In [23]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [24]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [25]:
# Create a pipeline for the transformed column here
my_new_column = Pipeline(steps=[('my_new_column', FunctionTransformer(new_col)),
                               ('scaler', StandardScaler())])


In [26]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        
        ######## ENTER THE TRANSFORMER FOR THE NEW COLUMN HERE ###########
        ('trans', my_new_column, feat_eng_columns)   
        ],   
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [27]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

array([[-0.31412013, -1.30301181, -0.86148829, ...,  0.        ,
         0.        , -0.42345858],
       [ 0.45733454,  0.7398414 ,  0.58636192, ...,  0.        ,
         0.        , -0.42345858],
       [-1.10330939,  0.2001631 ,  0.18146318, ...,  0.        ,
         0.        , -0.42345858],
       ...,
       [-0.21657988, -0.83000156, -0.82081329, ...,  0.        ,
         0.        , -0.42345858],
       [-0.46486414,  1.79196675,  1.36974799, ...,  0.        ,
         0.        , -0.42345858],
       [-0.31412013, -0.08740643, -0.21782887, ...,  0.        ,
         0.        , -0.42345858]])

In [28]:
train.shape

(1750, 13)

# Tranform: transform() for TEST

In [29]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

array([[ 0.06717356,  0.36706438,  0.32127798, ...,  0.        ,
         0.        , -0.42345858],
       [ 0.32432512,  0.57631513,  0.42769944, ...,  1.        ,
         0.        , -0.42345858],
       [-0.33185472,  0.41209537,  0.12227549, ...,  0.        ,
         0.        , -0.42345858],
       ...,
       [-0.8993616 , -0.51071616, -0.32326299, ...,  0.        ,
         0.        ,  2.31481716],
       [-0.5446698 , -0.82706576, -0.8366813 , ...,  0.        ,
         0.        , -0.42345858],
       [-0.5446698 , -0.06422056, -0.11380525, ...,  0.        ,
         0.        , -0.42345858]])

In [30]:
test_x.shape

(750, 21)

# Calculate the Baseline

In [35]:
# Find percentage
train_target.value_counts()/len(train_target)

0    0.603429
1    0.396571
Name: BAD, dtype: float64

# Train a shallow (one-layer) NN model

In [37]:
from sklearn.neural_network import MLPClassifier

#Default settings create 1 hidden layer with 100 neurons
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,))

mlp_clf.fit(train_x, train_target)

c:\users\rishabh\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier()

### Calculate the accuracy

In [39]:
from sklearn.metrics import accuracy_score
#Predict the train values
train_y_pred = mlp_clf.predict(train_x)

#Train accuracy
accuracy_score(train_target, train_y_pred)



0.9108571428571428

In [41]:
#Predict the test values
test_y_pred = mlp_clf.predict(test_x)

#Test accuracy
accuracy_score(test_target, test_y_pred)



0.832

In [43]:
from sklearn.metrics import confusion_matrix

#We usually create the confusion matrix on test set
confusion_matrix(test_target, test_y_pred)

array([[377,  56],
       [ 70, 247]], dtype=int64)

# Train a deep (multi-layered) NN model 

In [44]:
dnn_clf = MLPClassifier(hidden_layer_sizes=(50,25,10),
                       max_iter=1000)

dnn_clf.fit(train_x, train_target)



MLPClassifier(hidden_layer_sizes=(50, 25, 10), max_iter=1000)

In [45]:
#Let's check the number of iterations:
dnn_clf.n_iter_



333

In [46]:
#Let's check the number of layers:
dnn_clf.n_layers_

5

### Calculate the accuracy

In [48]:
#Predict the train values
train_y_pred = dnn_clf.predict(train_x)

#Train accuracy
accuracy_score(train_target, train_y_pred)

1.0

In [50]:
#Predict the test values
test_y_pred = dnn_clf.predict(test_x)

#Test accuracy
accuracy_score(test_target, test_y_pred)

0.836

# Optional: try grid search